In [2]:
import pyodbc
from random import randrange
from faker import Faker
from faker_marketdata import MarketDataProvider
from faker.providers import DynamicProvider

fake = Faker()
fake.add_provider(MarketDataProvider)

# cantidad de registros a generar
num_records = 50

# preparo conexión a la BD
conn = pyodbc.connect('DRIVER={SQL Server};SERVER=MYSERVER\SQLEXPRESS;DATABASE=metastock;UID=sa;PWD=somepass')

In [3]:
# emisores ya persistidos (para mantener integridad de datos)
cursor = conn.cursor()
issuers = cursor.execute("SELECT issuer_id FROM issuers;").fetchall()
cursor.close()

issuer_provider = DynamicProvider(
     provider_name = "issuer",
     elements = [id.rstrip() for id, in issuers],
)
fake.add_provider(issuer_provider)

In [ ]:
# script para la tabla SECURITIES

print("INSERT INTO dbo.securities (isin_code, ticker, name, issuer, issue_date, issue_amount) VALUES ")
for i in range(num_records):

    isin_code = fake.unique.isin()
    ticker = fake.unique.ticker()
    name = fake.name()
    issuer = fake.issuer()
    issue_date = fake.date()
    issue_amount = fake.random_int(min = 10000000, max = 100000000, step = 1000000)
    
    separator = ';' if (i == num_records-1) else ','

    print(f"\t('{isin_code}', '{ticker}', '{name}', '{issuer}', '{issue_date}', {issue_amount}){separator}")

In [5]:
# monedas ya persistidos (para mantener integridad de datos)
cursor = conn.cursor()
currencies = cursor.execute("SELECT iso_code FROM currencies;").fetchall()
cursor.close()

currency_provider = DynamicProvider(
     provider_name = "currency",
     elements = [code.rstrip() for code, in currencies],
)
fake.add_provider(currency_provider)

In [6]:
# productos ya persistidos (para mantener integridad de datos)
cursor = conn.cursor()
securities = cursor.execute("SELECT ticker FROM securities;").fetchall()
cursor.close()

security_provider = DynamicProvider(
     provider_name = "security",
     elements = [ticker.rstrip() for ticker, in securities],
)
fake.add_provider(security_provider)

In [7]:
# operadores ya persistidos (para mantener integridad de datos)
cursor = conn.cursor()
operators = cursor.execute("SELECT username FROM operators;").fetchall()
cursor.close()

operator_provider = DynamicProvider(
     provider_name = "operator",
     elements = [username.rstrip() for username, in operators],
)
fake.add_provider(operator_provider)

In [ ]:
# script para la tabla TRADES
import uuid

print("INSERT INTO dbo.trades (trade_id, agreement_date, quantity, security, price, currency, operator) VALUES ")

for i in range(num_records):

    trade_id = uuid.uuid4()
    agreement_date = fake.date_between(start_date = '-5y', end_date = 'today')
    quantity = fake.random_int(min = 10, max = 1000000, step = 5)
    security = fake.security()
    price = fake.random_int(min = 5, max = 1000) / 100
    currency = fake.currency()
    operator = fake.operator()
    
    separator = ';' if (i == num_records-1) else ','

    print(f"\t('{trade_id}', '{agreement_date}', {quantity}, '{security}', {price}, '{currency}', '{operator}'){separator}")

In [9]:
conn.close()